## Regression part b

Implement two-level cross-validation (see algorithm 6 of the lecture notes). We will use
2-level cross-validation to compare the models with K1 = K2 = 10 folds
. As a baseline
model, we will apply a linear regression model with no features, i.e. it computes the mean
of y on the training data, and use this value to predict y on the test data.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import sklearn.model_selection 
from sklearn.linear_model import Ridge
#from toolbox_02450 import train_neural_net, draw_neural_net, visualize_decision_boundary,correlated_ttest
import torch
from scipy import stats
from Data_preprocessing import *


In [4]:
df

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
457,214,0.40,5.98,31.72,0,64,28.45,0.00,58,0
458,182,4.20,4.41,32.10,0,52,28.61,18.72,52,1
459,108,3.00,1.59,15.23,0,40,20.09,26.64,55,0
460,118,5.40,11.61,30.79,0,64,27.35,23.97,40,0


In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn.linear_model as lm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.io import loadmat

# Load dataset
attribute_names = ['sbp','tobacco','chd','adiposity','typea','obesity','alcohol','age','famhist']
class_name      = ['ldl']

# Adjust X and y to keep selected attributes
X = df[attribute_names].values.astype(np.float32)
y = df[class_name].values.astype(np.float32).ravel()

# Compute dataset properties
N, M = X.shape  # Number of observations and attributes

# Standardize data
X_standardized = zscore(X, ddof=1)

# Overwrite X with standardized data as we will only use standardized data in the following
X = X_standardized

# Define K1 (outer CV) and K2 (inner CV)
K1, K2 = 10, 10
outer_cv = KFold(n_splits=K1, shuffle=True, random_state=42)

# Hyperparameter search space
lambda_values = [0.01, 0.1, 1, 10, 100]
h_values = [1, 5, 10, 20]

# Neural Network Model
def create_ann(input_size, hidden_units):
    model = nn.Sequential(
        nn.Linear(input_size, hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units, 1)
    )
    return model

# Initialize performance tracking
Error_train_baseline = np.empty(K1)
Error_test_baseline = np.empty(K1)
Error_train_lr = np.empty(K1)
Error_test_lr = np.empty(K1)
Error_train_ann = np.empty(K1)
Error_test_ann = np.empty(K1)

for k, (train_index, test_index) in enumerate(outer_cv.split(X)):
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]
    
    # Baseline Model (No Features)
    y_pred_baseline = np.full_like(y_test, y_train.mean())
    Error_train_baseline[k] = np.mean((y_train - y_train.mean()) ** 2)
    Error_test_baseline[k] = np.mean((y_test - y_pred_baseline) ** 2)
    
    # Inner cross-validation for hyperparameter tuning
    inner_cv = KFold(n_splits=K2, shuffle=True, random_state=42)
    best_lambda, best_h = None, None
    min_error_lr, min_error_ann = float("inf"), float("inf")
    
    for train_inner_idx, val_inner_idx in inner_cv.split(X_train):
        X_train_inner, X_val_inner = X_train[train_inner_idx], X_train[val_inner_idx]
        y_train_inner, y_val_inner = y_train[train_inner_idx], y_train[val_inner_idx]
        
        # Standardize data
        scaler = StandardScaler().fit(X_train_inner)
        X_train_inner_scaled = scaler.transform(X_train_inner)
        X_val_inner_scaled = scaler.transform(X_val_inner)
        
        # Ridge Regression Hyperparameter Tuning
        for lambda_ in lambda_values:
            model_lr = lm.Ridge(alpha=lambda_).fit(X_train_inner_scaled, y_train_inner)
            val_error = np.mean((y_val_inner - model_lr.predict(X_val_inner_scaled)) ** 2)
            if val_error < min_error_lr:
                min_error_lr = val_error
                best_lambda = lambda_
        
        # ANN Hyperparameter Tuning
        for h in h_values:
            model_ann = create_ann(M, h)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model_ann.parameters(), lr=0.01)
            
            X_train_tensor = torch.tensor(X_train_inner_scaled, dtype=torch.float32)
            y_train_tensor = torch.tensor(y_train_inner.reshape(-1, 1), dtype=torch.float32)
            X_val_tensor = torch.tensor(X_val_inner_scaled, dtype=torch.float32)
            y_val_tensor = torch.tensor(y_val_inner.reshape(-1, 1), dtype=torch.float32)
            
            for epoch in range(100):
                optimizer.zero_grad()
                y_pred = model_ann(X_train_tensor)
                loss = criterion(y_pred, y_train_tensor)
                loss.backward()
                optimizer.step()
            
            with torch.no_grad():
                val_pred = model_ann(X_val_tensor)
                val_error = criterion(val_pred, y_val_tensor).item()
            
            if val_error < min_error_ann:
                min_error_ann = val_error
                best_h = h
    
    # Train final models using the best hyperparameters
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model_lr = lm.Ridge(alpha=best_lambda).fit(X_train_scaled, y_train)
    Error_train_lr[k] = np.mean((y_train - model_lr.predict(X_train_scaled)) ** 2)
    Error_test_lr[k] = np.mean((y_test - model_lr.predict(X_test_scaled)) ** 2)
    
    model_ann = create_ann(M, best_h)
    optimizer = optim.Adam(model_ann.parameters(), lr=0.01)
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)
    
    for epoch in range(100):
        optimizer.zero_grad()
        y_pred = model_ann(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        test_pred = model_ann(X_test_tensor)
        Error_train_ann[k] = criterion(model_ann(X_train_tensor), y_train_tensor).item()
        Error_test_ann[k] = criterion(test_pred, y_test_tensor).item()
    
    print(f"Outer fold {k+1}/{K1} completed.")

# Print results
print("\nFinal Results:")
print(f"Baseline Model: Train Error = {Error_train_baseline.mean():.4f}, Test Error = {Error_test_baseline.mean():.4f}")
print(f"Ridge Regression: Train Error = {Error_train_lr.mean():.4f}, Test Error = {Error_test_lr.mean():.4f}")
print(f"ANN Model: Train Error = {Error_train_ann.mean():.4f}, Test Error = {Error_test_ann.mean():.4f}")


Outer fold 1/10 completed.
Outer fold 2/10 completed.
Outer fold 3/10 completed.
Outer fold 4/10 completed.
Outer fold 5/10 completed.
Outer fold 6/10 completed.
Outer fold 7/10 completed.
Outer fold 8/10 completed.
Outer fold 9/10 completed.
Outer fold 10/10 completed.

Final Results:
Baseline Model: Train Error = 4.2790, Test Error = 4.2914
Ridge Regression: Train Error = 3.2964, Test Error = 3.4238
ANN Model: Train Error = 3.3073, Test Error = 3.9032


In [ ]:
""" 
In this section, we will compare three models: the regularized linear regression model from the previous section, 
an artificial neural network (ANN) and a baseline. 
We are interested in two questions: Is one model better than the other? Is either model better than
 a trivial baseline?. We will attempt to answer these questions with two-level cross-validation.

 1. Implement two-level cross-validation (see algorithm 6 of the lecture notes). We will use
 2-level cross-validation to compare the models with K1 = K2 = 10 folds. As a baseline model, we will apply a linear regression model with no features, i.e. it computes the mean
 of y on the training data, and use this value to predict y on the test data.

 Make sure you can fit an ANN model to the data. As complexity-controlling parameter
 for the ANN, we will use the number of hidden units h. Based on a few test-runs, select
 a reasonable range of values for h (which should include h = 1), and describe the range of
 values you will use for h and lambda.
 """


from dtuimldmtools import (
    draw_neural_net,
    train_neural_net,
    visualize_decision_boundary,
)

attribute_names = ['sbp','tobacco','chd','adiposity','typea','obesity','alcohol','age','famhist']
class_name      = ['ldl']

# Adjust X and y to keep selected attributes
X = df[attribute_names].values.astype(np.float32)
y = df[class_name].values.astype(np.float32).ravel()

# Compute dataset properties
N, M = X.shape  # Number of observations and attributes

# Standardize data
X_standardized = zscore(X, ddof=1)

# Overwrite X with standardized data as we will only use standardized data in the following
X = X_standardized


# Define K1 (outer CV) and K2 (inner CV) folds
K1 = 10 # Number of outer loops
K2 = 10 # Number of inner loops
outer_cv = sklearn.model_selection.KFold(n_splits=K1, shuffle=True, random_state=42)

# Define ANN hyperparameters
min_n_hidden_units = 1  # Minimum number of hidden units
max_n_hidden_units = 5  # Maximum number of hidden units
max_iter = 10000  # Maximum number of iterations for training
loss_fn   = torch.nn.MSELoss()

# Hyperparameters search space 
lambda_values = [0.01, 0.1, 1, 10, 100] 
h_values = [1, 5, 10, 20]

# Define lists to store outer CV results
error_train_baseline = np.empty(K1)
error_test_baseline = np.empty(K1)
error_test_rlr = np.empty(K1)
error_test_ann = np.empty(K1)
best_lamdas = []
best_hs = []
results_table = []


# Loop over outer folds
for k, (train_index, test_index) in enumerate(outer_cv.split(X)):
    print(f"Outer fold {k+1}/{K1}")

    # Extract training and test set for current CV fold
    X_train_outer = X[train_index,:]
    y_train_outer = y[train_index]
    X_test_outer = X[test_index,:]
    y_test_outer = y[test_index]
  

    # Baseline model: mean of y_train (no features)
    y_pred_baseline = np.full_like(y_test_outer, np.mean(y_train_outer))
    error_train_baseline[k] =  np.square(y_train_outer - y_pred_baseline).sum() / len(y_train_outer)
    error_test_baseline[k] = np.square(y_test_outer - y_pred_baseline).sum() / len(y_test_outer)
    
    
    # Inner cross-validation for RLR and ANN hyperparameter tuning
    inner_cv = sklearn.model_selection.KFold(n_splits=K2, shuffle=True, random_state=42)
    best_lambda = None
    best_h = None
    min_error_rlr = np.inf
    min_error_ann = np.inf

    # Loop over inner folds
    for train_inner_index, val_index in inner_cv.split(X_train_outer):

        # Extract training and validation set for current CV fold
        X_train_inner = X_train_outer[train_inner_index,:]
        y_train_inner = y_train_outer[train_inner_index]
        X_val = X_train_outer[val_index,:]
        y_val = y_train_outer[val_index]
      


        # Hyperparameter tuning for RLR
        for lambda_value in lambda_values:
        
            model_rlr = Ridge(alpha=lambda_value)  # Define model with regularization parameter lambda
            model_rlr.fit(X_train_inner, y_train_inner)  # Train
            y_pred_rlr = model_rlr.predict(X_val)  # Predict validation set
            error_rlr = np.square(y_val - y_pred_rlr).sum() / len(y_val)  # Compute validation error

            if error_rlr < min_error_rlr:
                min_error_rlr = error_rlr
                best_lambda = lambda_value
        
        # Hyperparameter tuning for ANN

        # Convert data to PyTorch tensors
        X_train_inner_tensor = torch.tensor(X_train_inner, dtype=torch.float32)
        y_train_inner_tensor = torch.tensor(y_train_inner, dtype=torch.float32).view(-1, 1)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

        for n_hidden_units in range(min_n_hidden_units,max_n_hidden_units + 1):
            # Define ANN model
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n_hidden_units),  # M features to H hidden units
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.Tanh(),  # torch.nn.ReLU(),
                torch.nn.Linear(n_hidden_units, 1),  # H hidden units to 1 output neuron
                # No final transfer function, 
            )

            # Train the ANN model using the training data
            net, final_loss, learning_curve = train_neural_net(model, 
                                                               loss_fn,
                                                               X=X_train_inner_tensor, 
                                                               y=y_train_inner_tensor, 
                                                               n_replicates=1,
                                                               max_iter=max_iter)

            # Determine estimated y values for the validation set
            y_est_inner = net(X_val_tensor).detach().numpy()

            # Compute validation error
            error_ann = np.square(y_val - y_est_inner).sum() / len(y_val)

            # Check if the current error is less than the minimum error
            if error_ann < min_error_ann:
                min_error_ann = error_ann
                best_h = n_hidden_units
                best_model = net


       
        




Fold 1: Best Lambda for RLR: 100
Fold 2: Best Lambda for RLR: 100
Fold 3: Best Lambda for RLR: 0.01
Fold 4: Best Lambda for RLR: 100
Fold 5: Best Lambda for RLR: 1
Fold 6: Best Lambda for RLR: 0.01
Fold 7: Best Lambda for RLR: 100
Fold 8: Best Lambda for RLR: 0.01
Fold 9: Best Lambda for RLR: 100
Fold 10: Best Lambda for RLR: 100


In [8]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
from dtuimldmtools import train_neural_net
import warnings
from sklearn.exceptions import ConvergenceWarning

# Attribute selection
attribute_names = ['sbp','tobacco','chd','adiposity','typea','obesity','alcohol','age','famhist']
class_name = ['ldl']

# Prepare data
X = df[attribute_names].values.astype(np.float32)
y = df[class_name].values.astype(np.float32).ravel()
N, M = X.shape

# Standardize
X = zscore(X, ddof=1)

# Cross-validation setup
K1 = 10
K2 = 5
outer_cv = KFold(n_splits=K1, shuffle=True, random_state=42)

# Hyperparameter ranges
lambda_values = [0.01, 0.1, 1, 10, 100]
h_values = [1, 2, 3, 5]
max_iter = 2000
loss_fn = torch.nn.MSELoss()

# Storage
error_train_baseline = np.empty(K1)
error_test_baseline = np.empty(K1)
error_test_rlr = np.empty(K1)
error_test_ann = np.empty(K1)
best_lamdas = []
best_hs = []
results_table = []

# Outer CV loop
for k, (train_index, test_index) in enumerate(outer_cv.split(X)):
    print(f"Outer fold {k+1}/{K1}")

    X_train_outer, y_train_outer = X[train_index], y[train_index]
    X_test_outer, y_test_outer = X[test_index], y[test_index]

    # --- Baseline model ---
    y_pred_baseline = np.full_like(y_test_outer, np.mean(y_train_outer))
    error_train_baseline[k] = np.square(y_train_outer - np.mean(y_train_outer)).sum() / len(y_train_outer)
    error_test_baseline[k] = np.square(y_test_outer - y_pred_baseline).sum() / len(y_test_outer)

    # --- Inner CV for hyperparameter tuning ---
    inner_cv = KFold(n_splits=K2, shuffle=True, random_state=42)
    best_lambda = None
    best_h = None
    min_error_rlr = np.inf
    min_error_ann = np.inf

    for train_inner_index, val_index in inner_cv.split(X_train_outer):
        X_train_inner = X_train_outer[train_inner_index]
        y_train_inner = y_train_outer[train_inner_index]
        X_val = X_train_outer[val_index]
        y_val = y_train_outer[val_index]

        # --- RLR tuning ---
        for lambda_value in lambda_values:
            model_rlr = Ridge(alpha=lambda_value)
            model_rlr.fit(X_train_inner, y_train_inner)
            y_pred_rlr = model_rlr.predict(X_val)
            error_rlr = np.square(y_val - y_pred_rlr).sum() / len(y_val)

            if error_rlr < min_error_rlr:
                min_error_rlr = error_rlr
                best_lambda = lambda_value

        # --- ANN tuning ---
        X_train_inner_tensor = torch.tensor(X_train_inner, dtype=torch.float32)
        y_train_inner_tensor = torch.tensor(y_train_inner, dtype=torch.float32).view(-1, 1)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

        for h in h_values:
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, h),
                torch.nn.Tanh(),
                torch.nn.Linear(h, 1),
            )

            with warnings.catch_warnings():
                warnings.simplefilter("ignore", ConvergenceWarning)
                net, final_loss, learning_curve = train_neural_net(
                    model, loss_fn,
                    X=X_train_inner_tensor,
                    y=y_train_inner_tensor,
                    n_replicates=1,
                    max_iter=max_iter
                )

            y_est_val = net(X_val_tensor).detach().numpy()
            error_ann = np.square(y_val - y_est_val).sum() / len(y_val)

            if error_ann < min_error_ann:
                min_error_ann = error_ann
                best_h = h
                best_ann_model = net

    # Save best hyperparameters
    best_lamdas.append(best_lambda)
    best_hs.append(best_h)

    # --- Train best models on outer training set ---
    # RLR
    final_rlr = Ridge(alpha=best_lambda)
    final_rlr.fit(X_train_outer, y_train_outer)
    y_test_pred_rlr = final_rlr.predict(X_test_outer)
    error_test_rlr[k] = np.square(y_test_outer - y_test_pred_rlr).sum() / len(y_test_outer)

    # ANN
    X_train_outer_tensor = torch.tensor(X_train_outer, dtype=torch.float32)
    y_train_outer_tensor = torch.tensor(y_train_outer, dtype=torch.float32).view(-1, 1)
    X_test_tensor = torch.tensor(X_test_outer, dtype=torch.float32)

    best_ann = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, best_h),
        torch.nn.Tanh(),
        torch.nn.Linear(best_h, 1),
    )

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        trained_ann, _, _ = train_neural_net(best_ann, loss_fn,
                                             X=X_train_outer_tensor,
                                             y=y_train_outer_tensor,
                                             n_replicates=1,
                                             max_iter=max_iter)

    y_test_pred_ann = trained_ann(X_test_tensor).detach().numpy()
    error_test_ann[k] = np.square(y_test_outer - y_test_pred_ann).sum() / len(y_test_outer)

    # --- Store results ---
    results_table.append({
        "Fold": k+1,
        "lambda*": best_lambda,
        "h*": best_h,
        "Baseline Test Error": error_test_baseline[k],
        "RLR Test Error": error_test_rlr[k],
        "ANN Test Error": error_test_ann[k]
    })


Outer fold 1/10

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	16.207216	0.00082922744
		2000	7.632791	0.0006451087
		Final loss:
		2000	7.632791	0.0006451087

	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\dtuimldmtools\models\nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	7.280691	0.0017139013
		2000	3.5814657	0.0001591438
		Final loss:
		2000	3.5814657	0.0001591438

	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\dtuimldmtools\models\nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	4.4360847	0.0010515479
		2000	3.3162155	0.00011149637
		Final loss:
		2000	3.3162155	0.00011149637

	Replicate: 1/1
		Iter	Loss			Rel. loss


...

c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\dtuimldmtools\models\nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	3.1219113	0.00048219305
		2000	2.6594005	8.668524e-05
		Final loss:
		2000	2.6594005	8.668524e-05

	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\dtuimldmtools\models\nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	8.110571	0.0010508022
		2000	4.2131815	0.00031453435
		Final loss:
		2000	4.2131815	0.00031453435


c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\dtuimldmtools\models\nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


In [9]:
print("--- Results ---")
for result in results_table:
    print(f"Fold {result['Fold']}:")
    print(f"  lambda*: {result['lambda*']}")
    print(f"  h*: {result['h*']}")
    print(f"  Baseline Test Error: {result['Baseline Test Error']}")
    print(f"  RLR Test Error: {result['RLR Test Error']}")
    print(f"  ANN Test Error: {result['ANN Test Error']}")

--- Results ---
Fold 1:
  lambda*: 100
  h*: 1
  Baseline Test Error: 2.8038408756256104
  RLR Test Error: 2.1936287879943848
  ANN Test Error: 170.8423614501953
Fold 2:
  lambda*: 100
  h*: 2
  Baseline Test Error: 4.23134183883667
  RLR Test Error: 3.319079875946045
  ANN Test Error: 229.40969848632812
Fold 3:
  lambda*: 100
  h*: 1
  Baseline Test Error: 5.703014373779297
  RLR Test Error: 4.96721076965332
  ANN Test Error: 295.3771057128906
Fold 4:
  lambda*: 100
  h*: 1
  Baseline Test Error: 2.57759428024292
  RLR Test Error: 1.8649108409881592
  ANN Test Error: 140.0723876953125
Fold 5:
  lambda*: 100
  h*: 2
  Baseline Test Error: 5.7758708000183105
  RLR Test Error: 4.7767767906188965
  ANN Test Error: 305.45343017578125
Fold 6:
  lambda*: 10
  h*: 1
  Baseline Test Error: 5.032508373260498
  RLR Test Error: 3.8410308361053467
  ANN Test Error: 264.9061279296875
Fold 7:
  lambda*: 100
  h*: 1
  Baseline Test Error: 3.5277955532073975
  RLR Test Error: 3.0595483779907227
  ANN 